#  Rating: Best Restaurant for first dates (based on Yelp Dataset in SQL)

## 1. Problem Statement
### Goal:
Develop a rating of restaurants for first dates, taking into account the following criteria:
- Atmosphere (romantic/cozy).
- High ratings.
- A large number of positive reviews.
- Flexible work hours.

## 2. Work steps
### Step 1: Filtering restaurants
#### Goal: Select businesses that are restaurants.
- Actions:
- Use the yelp_business table.
- Select rows where the categories column contains the value Restaurants.

In [ ]:
# SELECT business_id, name, city, state, latitude, longitude, stars, review_count, categories
# FROM yelp_business
# WHERE categories LIKE '%Restaurants%';

## Step 3: Filter by opening hours
### Goal: Select restaurants that are open in the evening (important for dates).
- Actions:
- Use the yelp_opening_hours and yelp_closing_hours tables.
- Identify restaurants that are open after 6:00 PM.

In [ ]:
# SELECT o.business_id
# FROM yelp_opening_hours o
# JOIN yelp_closing_hours c ON o.business_id = c.business_id
# WHERE 
#     (
#     STR_TO_DATE(o.Monday_Open, '%h:%i %p') < '18:00:00' AND STR_TO_DATE(c.Monday_Close, '%h:%i %p') > '22:00:00'
#     ) OR (
#     STR_TO_DATE(o.Tuesday_Open, '%h:%i %p') < '18:00:00' AND STR_TO_DATE(c.Tuesday_Close, '%h:%i %p') > '22:00:00'
#     ) OR (
#     STR_TO_DATE(o.Wednesday_Open, '%h:%i %p') < '18:00:00' AND STR_TO_DATE(c.Wednesday_Close, '%h:%i %p') > '22:00:00'
#     ) OR (
#     STR_TO_DATE(o.Thursday_Open, '%h:%i %p') < '18:00:00' AND STR_TO_DATE(c.Thursday_Close, '%h:%i %p') > '22:00:00'
#     ) OR (
#     STR_TO_DATE(o.Friday_Open, '%h:%i %p') < '18:00:00' AND STR_TO_DATE(c.Friday_Close, '%h:%i %p') > '22:00:00'
#     ) OR (
#     STR_TO_DATE(o.Saturday_Open, '%h:%i %p') < '18:00:00' AND STR_TO_DATE(c.Saturday_Close, '%h:%i %p') > '22:00:00'
#     ) OR (
#     STR_TO_DATE(o.Sunday_Open, '%h:%i %p') < '18:00:00' AND STR_TO_DATE(c.Sunday_Close, '%h:%i %p') > '22:00:00'
#     );

### Step 4: Analyze review text
#### Goal: Find mentions of dating and romance in reviews.
Steps:
- Use the yelp_reviews table.
- Analyze text data, searching for keywords: romantic, date, cozy, intimate.
- Count the number of such mentions for each business.

Use ETL_SQL2CSV_RomanticReview.ipynb

## Step 5: Analyze Tips
### Goal: Consider short tips that might mention a date.
Actions:
- Analyze the yelp_tips table, searching for the same keywords.

In [ ]:
# CREATE TABLE aggregated_tips AS
# SELECT business_id, COUNT(*) AS romantic_tips
# FROM yelp_tips_combined
# WHERE text LIKE '%romantic%'
# GROUP BY business_id;

## Step 6: Combine results
### Goal: Collect all the received metrics into one dataset.
Actions:
- Combine the results from the previous steps (JOIN).

In [ ]:
# CREATE TABLE final_table AS
# SELECT b.business_id, MAX(b.name) AS name, MAX(b.city) AS city, MAX(b.state) AS state, MAX(b.latitude) AS latitude, MAX(b.longitude) AS longitude, MAX(b.stars) AS stars, MAX(b.review_count) AS review_count, MAX(b.categories) AS categories,
# 	MAX(r.romantic_mentions) AS romantic_mentions, MAX(r.date_mentions) AS date_mentions, MAX(r.cozy_mentions) AS cozy_mentions, MAX(t.romantic_tips) AS romantic_tips,
#     MAX(oc.schedule) AS schedule
# FROM filtered_business b
# LEFT JOIN open_close oc ON b.business_id = oc.business_id
# LEFT JOIN yelp_review_mentions r ON b.business_id = r.business_id
# LEFT JOIN aggregated_tips t ON b.business_id = t.business_id
# GROUP BY b.business_id;

## Step 7: Calculating the final rating
### Goal: Create an integral rating of restaurants.
Calculate the rating, taking into account:
- Number of romantic mentions.
- Restaurant stars (stars).
- Number of reviews.

In [ ]:
# SELECT business_id,
# 	name, city, state, stars, review_count, 
#        (romantic_mentions + date_mentions + cozy_mentions + romantic_tips) AS romance_score,
#        stars * (romantic_mentions + romantic_tips) / (review_count + 1) AS final_score,
#        schedule
# FROM final_table
# ORDER BY final_score DESC;

### Зберігаємо в CSV - SQL2CSV_in_chanks.ipynb


### 3. Final result - Table with top restaurants for dates, where there are columns:

- Restaurant name.
- City.
- Overall rating.
- Romantic mentions.
- Working hours.
- Visualization:

CSV_to_Folium_BestFirstDateUSA.ipynb

Result is romantic_restaurants_yelp.html